In [21]:
import os 
import pandas as pd

In [22]:
data_path = "dataset/enron1"

In [23]:
ham = []
spam = []

In [24]:
for root, dirs, files in os.walk(data_path):
    #in subdirectory
    for file in files:
        with open(os.path.join(root, file), 'r', encoding='latin1') as f:
            content = f.read()
            #if in ham, append ham 
            if 'ham' in root:
                ham.append({'filename': file, 'content': content, 'label': 'ham'})
            #if in spam, append spam
            elif 'spam' in root:
                spam.append({'filename': file, 'content': content, 'label': 'spam'})


In [25]:
ham_df = pd.DataFrame(ham)
spam_df = pd.DataFrame(spam)
df = pd.concat([ham_df, spam_df], ignore_index = True)

In [26]:
print(df.head)

<bound method NDFrame.head of                             filename  \
0     1365.2000-06-16.farmer.ham.txt   
1     3560.2001-02-09.farmer.ham.txt   
2     0877.2000-04-06.farmer.ham.txt   
3     2937.2000-11-27.farmer.ham.txt   
4     1270.2000-06-07.farmer.ham.txt   
...                              ...   
5167     1505.2004-07-09.GP.spam.txt   
5168     2148.2004-09-13.GP.spam.txt   
5169     2406.2004-10-06.GP.spam.txt   
5170     1459.2004-06-30.GP.spam.txt   
5171     2030.2004-08-30.GP.spam.txt   

                                                content label  
0     Subject: revised sea robin availabilities effe...   ham  
1     Subject: re : january spot purchases - deals n...   ham  
2     Subject: re : buyback / deficiency deals works...   ham  
3     Subject: king ranch processed volumes at tailg...   ham  
4     Subject: confirming requisitions\nconfirming t...   ham  
...                                                 ...   ...  
5167  Subject: what she doesnt know sprig

In [30]:
import string 
import nltk
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
def preprocess(text): 
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.replace('\n', ' ')
    tokens = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    #stemmer = PorterStemmer()
    #stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
    
    return ' '.join(filtered_tokens)

In [31]:
df['preprocessed_content'] = df['content'].apply(preprocess)
print(df.iloc[4000]['preprocessed_content'])


subject 8 dear friend size 1 order confirmation order shipped january via fedex federal express tracking number 45954036 thank registering userid 56075519 learn make fortune ebay complete turnkey system software videos turorials clk information clilings


In [49]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer

def train(X, y, model):
    vectorizer = CountVectorizer()
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state= 42)
    X_train_vectorized = vectorizer.fit_transform(X_train)
    X_test_vectorized = vectorizer.transform(X_test) 
    model.fit(X_train_vectorized, y_train)
    print(f'Model trained. Model Accuracy: {model.score(X_test_vectorized, y_test)}')
    return model

In [50]:
model = DecisionTreeClassifier()
X = df['preprocessed_content']
y= df['label']
model = train(X, y, model)

Model trained. Model Accuracy: 0.9487922705314009
